In [26]:
import joblib
import numpy as np
import pandas as pd

"""
To be scale features:

['Temperature(F)','Distance(mi)','Humidity(%)','Pressure(in)','Visibility(mi)','Wind_Speed(mph)',
'Precipitation(in)','Start_Lng','Start_Lat','Year', 'Month','Weekday','Day','Hour','Minute']
"""

np.random.seed(44)
basic_info = pd.DataFrame({
    "Temperature(F)": [np.random.uniform(-10.0, 110.0)],
    "Distance(mi)": [np.random.uniform(0.1, 500.0)],
    "Humidity(%)": [np.random.uniform(10.0, 100.0)],
    "Pressure(in)": [np.random.uniform(28.0, 31.0)],
    "Visibility(mi)": [np.random.uniform(0.1, 10.0)],
    "Wind_Speed(mph)": [np.random.uniform(0.0, 50.0)],
    "Precipitation(in)": [np.random.uniform(0.0, 5.0)],
    "Start_Lng": [-80.256503],
    "Start_Lat": [25.756695]
})
time = pd.DataFrame({
    "Year": [np.random.randint(2016, 2023)],
    "Month": [np.random.randint(1, 13)],
    "Weekday": [np.random.randint(0, 7)],
    "Day": [np.random.randint(1, 32)],
    "Hour": [np.random.randint(0, 24)],
    "Minute": [np.random.randint(0, 60)],
})

features = ['Temperature(F)','Distance(mi)','Humidity(%)','Pressure(in)','Visibility(mi)','Wind_Speed(mph)',
'Precipitation(in)','Start_Lng','Start_Lat','Year', 'Month','Weekday','Day','Hour','Minute']

scaler = joblib.load("../models/scaler.joblib")
basic_and_time = pd.concat([basic_info, time], axis=1)
basic_and_time[features] = scaler.transform(basic_and_time[features])

# extract into 2 dataframes
# time is last 6 columns
"""basic info follow this order: 
    Start_Lat
    Start_Lng
    Distance(mi)
    Temperature(F)
    Humidity(%)
    Pressure(in)
    Visibility(mi)
    Wind_Speed(mph)
    Precipitation(in)
"""
basic_info = basic_and_time[['Start_Lat', 'Start_Lng', 'Distance(mi)', 'Temperature(F)', 'Humidity(%)', 
                             'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)']]
time = basic_and_time.iloc[:, -6:]

# road features: all of them are 0
road = pd.DataFrame({
    "Amenity": [0],
    "Bump": [0],
    "Crossing": [0],
    "Give_Way": [0],
    "Junction": [0],
    "No_Exit": [0],
    "Railway": [0],
    "Roundabout": [0],
    "Station": [0],
    "Stop": [0],
    "Traffic_Calming": [0],
    "Traffic_Signal": [0]
})


wind_columns = [
    "Wind_Direction_E",
    "Wind_Direction_N",
    "Wind_Direction_NE",
    "Wind_Direction_NW",
    "Wind_Direction_S",
    "Wind_Direction_SE",
    "Wind_Direction_SW",
    "Wind_Direction_Variable",
    "Wind_Direction_W"
]

# Generating a single one-hot encoded row
row = [0] * len(wind_columns)
index = np.random.randint(0, len(wind_columns))
row[index] = 1

# Creating a DataFrame
wind = pd.DataFrame([row], columns=wind_columns)

weather_columns = [
    "Weather_Condition_Cloudy",
    "Weather_Condition_Fog",
    "Weather_Condition_Hail",
    "Weather_Condition_Rain",
    "Weather_Condition_Sand",
    "Weather_Condition_Smoke",
    "Weather_Condition_Snow",
    "Weather_Condition_Thunderstorm",
    "Weather_Condition_Tornado",
    "Weather_Condition_Windy"
]

row = [0] * len(weather_columns)
index = np.random.randint(0, len(weather_columns))
row[index] = 1
weather = pd.DataFrame([row], columns=weather_columns)

twilight = pd.DataFrame({
    "Civil_Twilight_Night": [np.random.randint(0, 1)]
})

# CITY
city = pd.DataFrame({
    "City": ['Miami']
})
binary_encoder = joblib.load("../models/binary_encoder.joblib")
city_enc = binary_encoder.transform(city['City'])

data = pd.concat([basic_info, road, time, wind, weather, twilight, city_enc], axis=1)

In [27]:
data

,Start_Lat,Start_Lng,Distance(mi),Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Amenity,...,City_4,City_5,City_6,City_7,City_8,City_9,City_10,City_11,City_12,City_13
0,0.048496,0.781134,0.33532,0.78245,0.767855,0.935984,0.030476,0.131301,0.197087,0,...,0,0,0,0,0,0,0,0,1,1


In [19]:
# load the model
model = joblib.load("../models/random_forest_model.joblib")

In [28]:
data.columns = data.columns.astype(str)

In [29]:
# make a prediction
prediction = model.predict(data)
prediction

array([2], dtype=int64)